In [13]:
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Set the environment variable for the API key (if needed, LangChain often uses getenv directly)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import init_chat_model

# Initialize the model
llm = init_chat_model("openai:gpt-5")

# To display the llm object (as implied by the final line "llm")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000002498B264B90> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002498B264550> root_client=<openai.OpenAI object at 0x000002498B265090> root_async_client=<openai.AsyncOpenAI object at 0x000002498B264690> model_name='gpt-5' model_kwargs={} openai_api_key=SecretStr('**********')


In [14]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000002498B265590>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002498B265D10>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [15]:
## Document Ingestion And Processing
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict

In [16]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [17]:
## chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
all_splits

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refi

In [18]:
## Vector Store\n",
from langchain_community.vectorstores import FAISS
vector_store=FAISS.from_documents(
    documents=all_splits,
    embedding=embeddings)
print(f"Vector store created with {vector_store.index.ntotal} vectors")

Vector store created with 63 vectors


In [19]:
from langchain.agents import tool

@tool()
def retrieve(query: str):
    """Retrieve the information related to the query"""
    
    retrieved_docs = vector_store.similarity_search(query, k=2)
    
    # Serialize the documents into a single string
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    
    return serialized, retrieved_docs

In [20]:
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition

In [21]:
# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    
    # Bind the retrieve tool to the language model
    llm_with_tools = llm.bind_tools([retrieve])
    
    # Invoke the LLM with the current list of messages
    response = llm_with_tools.invoke(state["messages"])
    
    # MessagesState appends messages to state instead of overwriting
    # Return the new AIMessage in a list under the "messages" key
    return {"messages": [response]}

In [22]:
# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])
tools

tools(tags=None, recurse=True, explode_args=False, func_accepts={'config': ('N/A', <class 'inspect._empty'>), 'store': ('store', None)}, tools_by_name={'retrieve': StructuredTool(name='retrieve', description='Retrieve the information related to the query', args_schema=<class 'langchain_core.utils.pydantic.retrieve'>, func=<function retrieve at 0x000002498B48C0E0>)}, tool_to_state_args={'retrieve': {}}, tool_to_store_arg={'retrieve': None}, handle_tool_errors=True, messages_key='messages')

In [23]:
# Assuming SystemMessage, llm, and MessagesState are imported/defined

# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    
    # 1. Extract recent ToolMessages (which contain the retrieved content string)
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        # Stop once we hit a non-tool message (the AI call that requested the tool)
        else:
            break
            
    # Reverse to process in chronological order
    tool_messages = recent_tool_messages[::-1]

    # 2. Format the retrieved content into a system prompt
    
    # FIX: ToolMessage content is the retrieved STRING. Access it as '.content'.
    docs_content = "\n\n".join(msg.content for msg in tool_messages)
    
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    
    # 3. Build the final prompt (SystemMessage + Conversation History)
    
    # Filter messages to include Human/System messages, and AI messages that *didn't* call a tool
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    
    # Create the final prompt with the RAG context
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # 4. Run the final LLM invocation
    response = llm.invoke(prompt)
    
    # Return the new AI message to update the state
    return {"messages": [response]}

In [24]:
from langgraph.graph import StateGraph, END
# Assuming MessagesState, query_or_respond, tools, generate, 
# tools_condition, and MemorySaver are imported/defined.

# Build graph
graph_builder = StateGraph(MessagesState)

# Corrected Node Definitions: Must provide (name, function)
graph_builder.add_node("query_or_respond", query_or_respond)
graph_builder.add_node("tools", tools)
graph_builder.add_node("generate", generate)

# Set the starting node
graph_builder.set_entry_point("query_or_respond")

# Conditional edge from the initial query/response node
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {
        END: END, # If tools_condition returns END, stop
        "tools": "tools", # If tools_condition returns "tools", go to the 'tools' node
    },
)

# Direct edges
graph_builder.add_edge("tools", "generate")

# Direct edge to end the workflow after generation
graph_builder.add_edge("generate", END)

# Compile the graph with memory
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# graph # (To display the object in a notebook environment)

In [25]:
# Specify an ID for the thread\n",
config = {"configurable": {"thread_id": "abc123"}}

In [26]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI 

# Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import init_chat_model

# Initialize the model
llm = init_chat_model("openai:gpt-5")

In [27]:
from langchain_core.messages import HumanMessage # Assuming this is used for input

input_message = "Hello"

# Use HumanMessage for explicit LangChain message object
user_message = HumanMessage(content=input_message)

# config needs to be defined (e.g., {"configurable": {"session_id": "..."}})
# Assuming 'config' is already defined in your scope

for step in graph.stream(
    {"messages": [user_message]}, # Use the proper message object
    stream_mode="values",
    config=config,
):
    # Print the last message in the list of messages updated in the current step
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hello
================================== Ai Message ==================================

Hello! How can I help you today?


In [28]:
vector_store.similarity_search("What is TAsk Decomposition")

[Document(id='7fd3f26a-6263-427a-99bc-981f28ba1e0d', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-f

In [29]:
from langchain_core.messages import HumanMessage
# Assuming 'graph' and 'config' are defined in the scope

input_message = "What is Task Decomposition?"

# Standardize the input message using a LangChain object
user_message = HumanMessage(content=input_message)

for step in graph.stream(
    {"messages": [user_message]},
    stream_mode="values",
    config=config,
):
    # Print the last message produced in the current step
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is Task Decomposition?
================================== Ai Message ==================================

Task decomposition is the practice of breaking a complex goal into smaller, clearer, and manageable subtasks. It’s used in project management, software engineering, AI, and everyday planning to improve clarity, estimation, execution, and quality.

Why it helps
- Clarifies scope and desired outcomes
- Improves estimates and scheduling
- Enables parallel work and specialization
- Reduces risk and makes progress trackable
- Eases testing, review, and reuse

Common techniques
- Work Breakdown Structure (WBS), Hierarchical Task Analysis
- User stories/epics and story mapping
- Checklists, flowcharts, SOPs
- Critical Path/PERT for dependencies and timing
- In AI: planner–executor, ReAct, self-ask, outline-then-detail

How to do it
1) Define the goal, constraints, success criteria, and deliverables.
2) S

In [30]:
from langchain_core.messages import HumanMessage
# Assuming 'graph' and 'config' are defined in the scope

input_message = "Can you look up some common ways of doing it?"

# Standardize the input message using a LangChain object
user_message = HumanMessage(content=input_message)

for step in graph.stream(
    {"messages": [user_message]},
    stream_mode="values",
    config=config,
):
    # Print the last message produced in the current step
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up some common ways of doing it?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_UuFlz6jc1SygzXWbDj2ZaAhH)
 Call ID: call_UuFlz6jc1SygzXWbDj2ZaAhH
  Args:
    query: common ways of task decomposition methods techniques: Work Breakdown Structure (WBS), Hierarchical Task Analysis (HTA), Goal-Directed Task Analysis (GDTA), GOMS, user stories/epics, story mapping, use-case decomposition, functional decomposition, object-oriented decomposition, divide-and-conquer, process decomposition (SIPOC, value stream mapping), dependency mapping (PERT/CPM), checklists/SOPs, mind mapping, AI methods (Chain of Thought, ReAct, Tree of Thoughts, planner-executor), tree decomposition, outline-then-detail. Provide brief descriptions and when to use.
================================= Tool Message =================================
Name: retrieve

('Source

In [31]:
### Conversation History\n",
chat_history=graph.get_state(config).values["messages"]
for message in chat_history:
       message.pretty_print()

================================ Human Message =================================

Hello
================================== Ai Message ==================================

Hello! How can I help you today?
================================ Human Message =================================

What is Task Decomposition?
================================== Ai Message ==================================

Task decomposition is the practice of breaking a complex goal into smaller, clearer, and manageable subtasks. It’s used in project management, software engineering, AI, and everyday planning to improve clarity, estimation, execution, and quality.

Why it helps
- Clarifies scope and desired outcomes
- Improves estimates and scheduling
- Enables parallel work and specialization
- Reduces risk and makes progress trackable
- Eases testing, review, and reuse

Common techniques
- Work Breakdown Structure (WBS), Hierarchical Task Analysis
- User stories/epics and story mapping
- Checklists, flowcharts, SO

In [32]:
### ReAct Agent Architecture-Persistant Memory

from langgraph.prebuilt import create_react_agent
memory = MemorySaver()
agent_executor=create_react_agent(llm,[retrieve],checkpointer=memory)

In [33]:
config = {"configurable": {"thread_id": "def234"}}

In [34]:
input_message = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

In [35]:
for event in agent_executor.stream(
    {"messages": [user_message]},
    stream_mode="values",
    config=config
):
    # Print the last message produced in the current step
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up some common ways of doing it?
================================== Ai Message ==================================

I’m happy to help—what is “it” in this case? Tell me the task or topic you have in mind, and I’ll look up common approaches and summarize them.

Examples of what you might mean:
- Learning a skill (e.g., coding, a new language)
- Solving a technical task (e.g., deploying a web app, setting up 2FA)
- Career moves (e.g., switching fields, negotiating salary)
- Everyday tasks (e.g., budgeting, organizing a move, cooking a dish)

If you share your goal, constraints, and any tools or context, I can tailor the list and provide steps, pros/cons, and links.
